# Analyze NBA Matchup

## Module Imports and Settings

In [1]:
from bs4 import BeautifulSoup as Soup
from bs4 import Comment
from sys import exit
from os import path
import requests
from pandas import DataFrame
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

In [2]:
pd.options.display.max_columns = None
InteractiveShell.ast_node_interactivity = "all"

## Global Variables

In [3]:
DATA_DIR = 'C:\\Users\\Harry\\Documents\\LTCWFF\\ltcwff_files\\data'

teams = {'ATL': 'Atlanta Hawks', 'BOS': 'Boston Celtics', 'BRK': 'Brooklyn Nets', 'CHO': 'Charlotte Hornets', 'CHI': 'Chicago Bulls', 'CLE': 'Cleveland Cavaliers', 'DAL': 'Dallas Mavericks', 'DEN': 'Denver Nuggets', 'DET': 'Detroit Pistons', 'GSW': 'Golden State Warriors', 'HOU': 'Houston Rockets', 'IND': 'Indiana Pacers', 'LAC': 'Los Angeles Clippers', 'LAL': 'Los Angeles Lakers', 'MEM': 'Memphis Grizzlies', 'MIA': 'Miami Heat', 'MIL': 'Milwaukee Bucks', 'MIN': 'Minnesota Timberwolves', 'NOP': 'New Orleans Pelicans', 'NYK': 'New York Knicks', 'OKC': 'Oklahoma City Thunder', 'ORL': 'Orlando Magic', 'PHI': 'Philadelphia 76ers', 'PHO': 'Phoenix Suns', 'POR': 'Portland Trail Blazers', 'SAC': 'Sacramento Kings', 'SAS': 'San Antonio Spurs', 'TOR': 'Toronto Raptors', 'UTA': 'Utah Jazz', 'WAS': 'Washington Wizards'}

odds_teams = {'ATL': 'Atlanta', 'BOS': 'Boston', 'BRK': 'Brooklyn', 'CHO': 'Charlotte', 'CHI': 'Chicago', 'CLE': 'Cleveland', 'DAL': 'Dallas', 'DEN': 'Denver', 'DET': 'Detroit', 'GSW': 'Golden State', 'HOU': 'Houston', 'IND': 'Indiana', 'LAC': 'LA', 'LAL': 'L.A. Lakers', 'MEM': 'Memphis', 'MIA': 'Miami', 'MIL': 'Milwaukee', 'MIN': 'Minnesota', 'NOP': 'New Orleans', 'NYK': 'New York', 'OKC': 'Oklahoma City', 'ORL': 'Orlando', 'PHI': 'Philadelphia', 'PHO': 'Phoenix', 'POR': 'Portland', 'SAC': 'Sacramento', 'SAS': 'San Antonio', 'TOR': 'Toronto', 'UTA': 'Utah', 'WAS': 'Washington'}

months = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}

## Load Data

In [4]:
#odds = pd.read_csv(path.join(DATA_DIR, 'odds_data_formatted_2.csv'))
#odds = odds.drop(odds.columns[0], axis = 1)
#odds = odds.set_index('Date')
#odds

inv_odds_teams = {v: k for k, v in odds_teams.items()}

odds = pd.read_csv(path.join(DATA_DIR, 'scraped_odds_data_2021.csv'))
odds = odds.drop(odds.columns[0], axis = 1)
odds = odds.set_index('Date')
odds['Team'] = [ inv_odds_teams[team] if team != 'L.A. Clippers' else 'LAC' for team in odds['Team'] ]
odds

,Team,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20201222,GSW,7.5,-108,275,234.5,-109,234.5,-101,4.5,-110,195,120.0,-110,120.0,-110,-2.5,127.0,-135.0,116.0,105.0,116.0,-125.0,2.5,-105,170,59.5,-115,59.5,-105,1.5,-105,125,59.5,-110,59.5,-110,0.5,-135,-120.0,57.0,105,57.0,-125,1.0,-110,NaN,56.0,-110,56.0,-110
20201222,BRK,-7.5,-102,-305,234.5,-109,234.5,-101,-4.5,-110,-235,120.0,-110,120.0,-110,2.5,-147.0,115.0,116.0,105.0,116.0,-125.0,-2.5,-115,-200,59.5,-115,59.5,-105,-1.5,-115,-154,59.5,-110,59.5,-110,-0.5,115,100.0,57.0,105,57.0,-125,-1.0,-110,NaN,56.0,-110,56.0,-110
20201222,LAC,2.0,-107,115,219.0,-107,219.0,-103,1.0,-110,110,111.5,-110,111.5,-110,4.5,-135.0,145.0,109.5,105.0,109.5,-125.0,1.0,-110,115,56.0,-110,56.0,-110,0.5,-110,-103,56.0,-110,56.0,-110,2.0,-120,135.0,53.5,103,53.5,-123,0.5,-110,NaN,53.0,-110,53.0,-110
20201222,LAL,-2.0,-103,-125,219.0,-107,219.0,-103,-1.0,-110,-130,111.5,-110,111.5,-110,-4.5,115.0,-165.0,109.5,105.0,109.5,-125.0,-1.0,-110,-135,56.0,-110,56.0,-110,-0.5,-110,-120,56.0,-110,56.0,-110,-2.0,100,-155.0,53.5,103,53.5,-123,-0.5,-110,NaN,53.0,-110,53.0,-110
20201223,MIA,-4.5,-108,-185,219.0,-103,219.0,-107,-2.0,-110,-150,112.0,-110,112.0,-110,-5.5,102.0,-235.0,107.0,-102.0,107.0,-118.0,-1.0,-115,-140,55.5,-110,55.5,-110,-0.5,-115,-141,54.5,-110,54.5,-110,-2.0,-110,-170.0,53.0,-110,53.0,-110,-1.0,-115,NaN,52.0,-115,52.0,-105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20210211,DET,4.0,-105,160,214.5,-105,214.5,-105,2.0,-110,130,109.0,-110,109.0,-110,4.0,-123.0,165.0,105.0,-102.0,105.0,-118.0,1.0,-115,110,55.5,-110,55.5,-110,0.5,-105,105,54.0,-105,54.0,-115,2.5,-110,160.0,52.0,-110,52.0,-110,0.5,-105,NaN,52.5,-105,52.5,-115
20210211,PHI,-6.0,-105,-220,229.0,-105,229.0,-105,-3.0,-110,-170,117.0,-110,117.0,-110,-4.0,115.0,-175.0,111.5,-103.0,111.5,-117.0,-1.5,-110,-150,59.0,-105,59.0,-115,-1.0,-115,-149,54.0,-115,54.0,-105,4.5,-110,-155.0,56.0,-125,56.0,105,-1.0,-115,NaN,56.5,-115,56.5,-105
20210211,POR,6.0,-105,200,229.0,-105,229.0,-105,3.0,-110,150,117.0,-110,117.0,-110,4.0,-135.0,155.0,111.5,-103.0,111.5,-117.0,1.5,-110,130,59.0,-105,59.0,-115,1.0,-105,120,54.0,-115,54.0,-105,-4.5,-110,135.0,56.0,-125,56.0,105,1.0,-105,NaN,56.5,-115,56.5,-105


## Helper Functions

In [5]:
def get_url_from_team(team, year, games = '', prefix = 'https://www.basketball-reference.com/teams'):
    return f'{prefix}/{team}/{year}{games}.html'

In [6]:
def get_soup(team, year, games = ''):
    url = get_url_from_team(team, year, games)
    print(url)
    response = requests.get(url)
    if not 200 <= response.status_code < 300:
        exit('Invalid Team')
    return Soup(response.content, 'html.parser')

In [7]:
def parse_row(row):
    result = [ x.string for x in row.find_all('td') ]
    return result

In [8]:
def table_to_df(table, overheader = 0):
    cols = table.find('thead').find_all('tr')[overheader].find_all('th')
    cols = [ col.string for col in cols ]
    cols
    
    stat_table = table.find('tbody')
    stat_table
        
    rows = stat_table.find_all('tr')
    rows
    
    headers = [ row.find('th').string for row in rows ]
    headers = [ header for header in headers if header != 'G' ]
    
    list_of_parsed_rows = [ parse_row(row) for row in rows[0:len(rows)] ]
    list_of_parsed_rows = [ row for row in list_of_parsed_rows if row != [] ]
    list_of_parsed_rows
    
    df = DataFrame(list_of_parsed_rows)
    df.insert(0, '', headers)
    df.columns = cols
    
    return df

In [9]:
def convert_df_to_int(df):
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            continue
    return df

## Team Statistics

In [10]:
def get_team_misc(soup, team, year):    
    div = soup.find('div', {'id': 'all_team_misc'})
    comment = div.find(string = lambda text: isinstance(text, Comment))
    table = Soup(comment, 'html.parser')
    table

    return table_to_df(table, 1)

In [11]:
def get_team_stats(soup, team, year, prompt = 'Team to research: '):    
    div = soup.find('div', {'id': 'all_team_and_opponent'})
    comment = div.find(string = lambda text: isinstance(text, Comment))
    table = Soup(comment, 'html.parser')
    
    return table_to_df(table, 0)

In [12]:
def get_team_dfs(team, year = 2021):
    soup = get_soup(team, year)
    
    main_df = get_team_stats(soup, team, year)
    
    misc_df = get_team_misc(soup, team, year)
    
    return [main_df, misc_df]

## Game Statistics

In [13]:
def get_all_games(team, year = 2021):
    soup = get_soup(team, year, '_games')
    
    table = soup.find('table', {'id': 'games'})
    table
    
    tds = table.find_all('td', {'data-stat': 'box_score_text'})
    hrefs = [ f"https://www.basketball-reference.com{td.find('a')['href']}" for td in tds ]
        
    df = table_to_df(table)
    
    df['url'] = hrefs
    
    return df

In [14]:
def get_game_stats(url):
    response = requests.get(url)
    if not 200 <= response.status_code < 300:
        return None
    soup = Soup(response.content, 'html.parser')
    
    line_div = soup.find('div', {'id': 'all_line_score'})
    line_comment = line_div.find(string = lambda text: isinstance(text, Comment))
    line_table = Soup(line_comment, 'html.parser')
    
    factors_div = soup.find('div', {'id': 'all_four_factors'})
    factors_comment = factors_div.find(string = lambda text: isinstance(text, Comment))
    factors_table = Soup(factors_comment, 'html.parser')
    
    line_df = table_to_df(line_table, 1)
    line_df = line_df.set_index(line_df.columns[0])
    factors_df = table_to_df(factors_table, 1)
    factors_df = factors_df.set_index(factors_df.columns[0])
    
    df = pd.concat([line_df, factors_df], axis = 1)
    
    return df

## Analyze Matchup

In [15]:
team_1 = input('Team 1: ')
team_2 = input('Team 2: ')

In [16]:
team_1_dfs = get_team_dfs(team_1)
team_1_dfs[0]
team_1_dfs[1]

https://www.basketball-reference.com/teams/LAL/2021.html


,,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Team,30,7300,1265,2598,.487,323,902,.358,942,1696,.555,518,691,.750,288,1071,1359,743,214,182,455,554,3371
1,Team/G,None,243.3,42.2,86.6,.487,10.8,30.1,.358,31.4,56.5,.555,17.3,23.0,.750,9.6,35.7,45.3,24.8,7.1,6.1,15.2,18.5,112.4
2,Lg Rank,None,2,9,26,3,27,27,19,1,7,2,12,8,25,18,7,8,17,21,2,25,7,17
3,Year/Year,None,1.1%,-0.4%,-1.9%,+.007,-2.2%,-4.8%,+.009,0.2%,-0.3%,+.003,-2.7%,-5.4%,+.020,-10.0%,1.8%,-0.9%,-2.5%,-17.2%,-8.0%,-0.0%,-10.9%,-0.9%
4,Opponent,30,7300,1204,2674,.450,346,982,.352,858,1692,.507,422,553,.763,272,971,1243,742,251,124,435,653,3176
5,Opponent/G,None,243.3,40.1,89.1,.450,11.5,32.7,.352,28.6,56.4,.507,14.1,18.4,.763,9.1,32.4,41.4,24.7,8.4,4.1,14.5,21.8,105.9
6,Lg Rank,None,2,7,19,5,6,6,3,18,25,5,2,1,7,3,6,2,15,26,5,16,3,2
7,Year/Year,None,1.1%,2.8%,2.3%,+.002,-1.0%,-2.1%,+.004,4.3%,5.0%,-.003,-21.4%,-19.5%,-.018,-3.6%,-1.5%,-2.0%,5.7%,1.5%,11.6%,-8.9%,0.1%,-1.7%


,,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,Pace,FTr,3PAr,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA,Arena,Attendance
0,Team,22,8,21,9,6.50,-0.81,5.69,112.1,105.6,98.9,.266,.347,.549,13.6,22.9,.199,.515,13.0,79.7,.158,STAPLES Center,0
1,Lg Rank,2,29,3,4,4,27,4,16,1,19,7,25,8,26,14,11,4,14,3,4,None,12


In [17]:
team_2_dfs = get_team_dfs(team_2)
team_2_dfs[0]
team_2_dfs[1]

https://www.basketball-reference.com/teams/LAC/2021.html


,,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Team,30,7200,1254,2584,.485,430,1019,.422,824,1565,.527,525,624,.841,281,1039,1320,731,225,137,399,573,3463
1,Team/G,None,240.0,41.8,86.1,.485,14.3,34.0,.422,27.5,52.2,.527,17.5,20.8,.841,9.4,34.6,44.0,24.4,7.5,4.6,13.3,19.1,115.4
2,Lg Rank,None,3,10,28,4,7,19,1,20,20,17,10,22,1,20,16,17,19,14,20,5,10,5
3,Year/Year,None,-0.6%,0.6%,-3.5%,+.020,15.3%,1.5%,+.051,-5.7%,-6.5%,+.004,-15.9%,-20.9%,+.050,-12.1%,-6.4%,-7.7%,2.7%,5.7%,-2.7%,-8.9%,-13.7%,-0.8%
4,Opponent,30,7200,1213,2610,.465,354,999,.354,859,1611,.533,477,601,.794,280,963,1243,702,210,102,408,544,3257
5,Opponent/G,None,240.0,40.4,87.0,.465,11.8,33.3,.354,28.6,53.7,.533,15.9,20.0,.794,9.3,32.1,41.4,23.4,7.0,3.4,13.6,18.1,108.6
6,Lg Rank,None,3,10,9,15,7,10,7,20,15,17,8,7,27,7,3,2,6,7,1,20,29,5
7,Year/Year,None,-0.6%,2.3%,-3.5%,+.026,-4.1%,-6.6%,+.009,5.2%,-1.6%,+.034,-14.2%,-18.3%,+.038,-12.8%,-7.2%,-8.5%,-1.4%,-6.7%,-27.6%,-3.3%,-20.7%,-1.2%


,,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,Pace,FTr,3PAr,eFG%,TOV%,ORB%,FT/FGA,eFG%,TOV%,DRB%,FT/FGA,Arena,Attendance
0,Team,21,9,21,9,6.87,-0.90,5.97,118.3,111.3,97.5,.241,.394,.568,12.2,22.6,.203,.533,12.4,78.8,.183,STAPLES Center,0
1,Lg Rank,3,28,2,2,2,28,3,2,14,27,18,15,3,10,18,9,10,18,9,10,None,12


In [18]:
def dfs_to_composite_df(team, dfs):
    df_basic = dfs[0].iloc[[1], :]
    df_basic.iloc[0, 0] = f'{team}/G'
    df_basic = df_basic.set_index(df_basic.columns[0])
    df_basic_lg = dfs[0].iloc[[2], :]
    df_basic_lg.iloc[0, 0] = f'{team}/LG'
    df_basic_lg = df_basic_lg.set_index(df_basic_lg.columns[0])
    #print(df_basic_lg)
    df_advanced = dfs[1].iloc[[0], :]
    df_advanced.iloc[0, 0] = f'{team}/G'
    df_advanced = df_advanced.set_index(df_advanced.columns[0])
    df_advanced_lg = dfs[1].iloc[[1], :]
    df_advanced_lg.iloc[0, 0] = f'{team}/LG'
    df_advanced_lg = df_advanced_lg.set_index(df_advanced_lg.columns[0])
    #print(df_advanced_lg)
    row_1 = pd.concat([df_basic, df_advanced], axis = 1)
    row_2 = pd.concat([df_basic_lg, df_advanced_lg], axis = 1)
    #print(row_1)
    #print(row_2)
    composite_df = row_1.append(row_2)
    #print(type(list(composite_df.columns[0:38])))
    #print(list(composite_df.columns[0:38]) + [ f'OPP {col}' for col in composite_df.columns[38:] ])
    composite_df.columns = list(composite_df.columns[0:39]) + [ f'OPP {col}' for col in composite_df.columns[39:] ]
    return composite_df

In [19]:
composite_df = dfs_to_composite_df(team_1, team_1_dfs).append(dfs_to_composite_df(team_2, team_2_dfs))
composite_df = composite_df.drop(labels = ['G', 'OPP Arena', 'OPP Attendance'], axis = 1)
#composite_df = composite_df.iloc[:, 0:38]
composite_df.iloc[[0, 2], :] = convert_df_to_int(composite_df.iloc[[0, 2], :])

composite_df = composite_df.reindex([composite_df.index[0], composite_df.index[2], composite_df.index[1], composite_df.index[3]])
composite_df
#tm_2, lg_1 = composite_df.iloc[2].copy(), composite_df.iloc[1].copy()
#composite_df.iloc[1], composite_df.iloc[2] = tm_2, lg_1

,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,Pace,FTr,3PAr,eFG%,TOV%,ORB%,FT/FGA,OPP eFG%,OPP TOV%,OPP DRB%,OPP FT/FGA
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
LAL/G,243.3,42.2,86.6,0.487,10.8,30.1,0.358,31.4,56.5,0.555,17.3,23,0.75,9.6,35.7,45.3,24.8,7.1,6.1,15.2,18.5,112.4,22,8,21,9,6.5,-0.81,5.69,112.1,105.6,98.9,0.266,0.347,0.549,13.6,22.9,0.199,0.515,13,79.7,0.158
LAC/G,240,41.8,86.1,0.485,14.3,34,0.422,27.5,52.2,0.527,17.5,20.8,0.841,9.4,34.6,44,24.4,7.5,4.6,13.3,19.1,115.4,21,9,21,9,6.87,-0.9,5.97,118.3,111.3,97.5,0.241,0.394,0.568,12.2,22.6,0.203,0.533,12.4,78.8,0.183
LAL/LG,2,9,26,3,27,27,19,1,7,2,12,8,25,18,7,8,17,21,2,25,7,17,2,29,3,4,4,27,4,16,1,19,7,25,8,26,14,11,4,14,3,4
LAC/LG,3,10,28,4,7,19,1,20,20,17,10,22,1,20,16,17,19,14,20,5,10,5,3,28,2,2,2,28,3,2,14,27,18,15,3,10,18,9,10,18,9,10


In [20]:
means = composite_df.iloc[[0, 1], :].mean()
#means = composite_df.iloc[[0, 2], :].mean()
means.name = 'Mean'
composite_df.append(means)

,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,Pace,FTr,3PAr,eFG%,TOV%,ORB%,FT/FGA,OPP eFG%,OPP TOV%,OPP DRB%,OPP FT/FGA
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
LAL/G,243.3,42.2,86.6,0.487,10.8,30.1,0.358,31.4,56.5,0.555,17.3,23,0.75,9.6,35.7,45.3,24.8,7.1,6.1,15.2,18.5,112.4,22,8,21,9,6.5,-0.81,5.69,112.1,105.6,98.9,0.266,0.347,0.549,13.6,22.9,0.199,0.515,13,79.7,0.158
LAC/G,240,41.8,86.1,0.485,14.3,34,0.422,27.5,52.2,0.527,17.5,20.8,0.841,9.4,34.6,44,24.4,7.5,4.6,13.3,19.1,115.4,21,9,21,9,6.87,-0.9,5.97,118.3,111.3,97.5,0.241,0.394,0.568,12.2,22.6,0.203,0.533,12.4,78.8,0.183
LAL/LG,2,9,26,3,27,27,19,1,7,2,12,8,25,18,7,8,17,21,2,25,7,17,2,29,3,4,4,27,4,16,1,19,7,25,8,26,14,11,4,14,3,4
LAC/LG,3,10,28,4,7,19,1,20,20,17,10,22,1,20,16,17,19,14,20,5,10,5,3,28,2,2,2,28,3,2,14,27,18,15,3,10,18,9,10,18,9,10
Mean,241.65,42,86.35,0.486,12.55,32.05,0.39,29.45,54.35,0.541,17.4,21.9,0.7955,9.5,35.15,44.65,24.6,7.3,5.35,14.25,18.8,113.9,21.5,8.5,21,9,6.685,-0.855,5.83,115.2,108.45,98.2,0.2535,0.3705,0.5585,12.9,22.75,0.201,0.524,12.7,79.25,0.1705


In [21]:
games = get_all_games(team_1)
#head_to_heads = [ games.loc[ind, :] for ind in games.index if games.loc[ind, 'Opponent'] == teams[team_2] ]
head_to_heads = games.loc[games['Opponent'] == teams[team_2], :]
head_to_heads

https://www.basketball-reference.com/teams/LAL/2021_games.html


,G,Date,Start (ET),,,,Opponent,,,Tm,Opp,W,L,Streak,Notes,url
0,1,"Tue, Dec 22, 2020",10:00p,None,Box Score,None,Los Angeles Clippers,L,None,109,116,0,1,L 1,None,https://www.basketball-reference.com/boxscores...


In [22]:
# box_scores = [ get_game_stats(url) for url in games['url'] ]
# for box_score in box_scores:
#     box_score

for ind in head_to_heads.index:
    box_score = get_game_stats(head_to_heads.loc[ind, 'url'])
    #box_score.index.name = 'Team'
    full_date = head_to_heads.loc[ind, 'Date'].replace(',', '').split(' ')
    year = '2020' if months[full_date[1]] == 12 else '2021'
    #date = year + str(months[full_date[1]]) + str(full_date[2])
    date = f'{year}{months[full_date[1]]:02d}{int(full_date[2]):02d}'
    #if full_date[1] != 'Dec':
    #    continue
    #print(odds)
    if year == '2021' and (months[full_date[1]] > 1 or int(full_date[2]) > 24):
        continue
    all_games = odds.loc[date, :]
    game_odds = all_games.loc[(all_games['Team'] == team_1) | (all_games['Team'] == team_2)]
    game_odds = game_odds.set_index('Team')
    summary = pd.concat([box_score, game_odds], axis = 1)
    summary

    covered = float(summary.loc[summary.index[0], 'T']) - float(summary.loc[summary.index[1], 'T']) > -(summary.loc[summary.index[0], 'Spread'])
    covered


,1,2,3,4,T,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
LAC,39,17,33,27,116,103.5,.548,13.6,22.9,.151,112.1,2.0,-107,115,219.0,-107,219.0,-103,1.0,-110,110,111.5,-110,111.5,-110,4.5,-135.0,145.0,109.5,105.0,109.5,-125.0,1.0,-110,115,56.0,-110,56.0,-110,0.5,-110,-103,56.0,-110,56.0,-110,2.0,-120,135.0,53.5,103,53.5,-123,0.5,-110,NaN,53.0,-110,53.0,-110
LAL,19,35,24,31,109,103.5,.525,16.7,21.6,.296,105.3,-2.0,-103,-125,219.0,-107,219.0,-103,-1.0,-110,-130,111.5,-110,111.5,-110,-4.5,115.0,-165.0,109.5,105.0,109.5,-125.0,-1.0,-110,-135,56.0,-110,56.0,-110,-0.5,-110,-120,56.0,-110,56.0,-110,-2.0,100,-155.0,53.5,103,53.5,-123,-0.5,-110,NaN,53.0,-110,53.0,-110


True

In [23]:
covereds = []
overs = []

for ind in games.index:
    box_score = get_game_stats(games.loc[ind, 'url'])
    #box_score.index.name = 'Team'
    full_date = games.loc[ind, 'Date'].replace(',', '').split(' ')
    year = '2020' if months[full_date[1]] == 12 else '2021'
    #date = year + str(months[full_date[1]]) + str(full_date[2])
    date = f'{year}{months[full_date[1]]:02d}{int(full_date[2]):02d}'
    if year == '2021' and (months[full_date[1]] > 1 or int(full_date[2]) > 24):
        continue
    #print(odds)
    all_games = odds.loc[date, :]
    game_odds = all_games.loc[(all_games['Team'] == team_1)]
    game_odds = game_odds.set_index('Team')
    summary = pd.concat([box_score, game_odds], axis = 1)
    summary

    if summary.loc[summary.index[0], 'Spread'] != None:
        teams = list(summary.index)
        opponent = teams[1] if teams.index(team_1) == 0 else teams[0]
        covered = float(summary.loc[team_1, 'T']) - float(summary.loc[opponent, 'T']) > -(summary.loc[team_1, 'Spread'])
        covereds.append(covered)
        over = float(summary.loc[team_1, 'T']) + float(summary.loc[opponent, 'T']) > summary.loc[team_1, 'Over']
        overs.append(over)

f'{team_1} covered {sum(bool(x) for x in covereds)} out of {len(covereds)} games'
f'{team_1} hit the over on {sum(bool(x) for x in overs)} out of {len(overs)} games'

,1,2,3,4,T,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
LAC,39,17,33,27,116,103.5,.548,13.6,22.9,.151,112.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAL,19,35,24,31,109,103.5,.525,16.7,21.6,.296,105.3,-2.0,-103.0,-125.0,219.0,-107.0,219.0,-103.0,-1.0,-110.0,-130.0,111.5,-110.0,111.5,-110.0,-4.5,115.0,-165.0,109.5,105.0,109.5,-125.0,-1.0,-110.0,-135.0,56.0,-110.0,56.0,-110.0,-0.5,-110.0,-120.0,56.0,-110.0,56.0,-110.0,-2.0,100.0,-155.0,53.5,103.0,53.5,-123.0,-0.5,-110.0,NaN,53.0,-110.0,53.0,-110.0


,1,2,3,4,T,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
DAL,30,27,28,30,115,98.9,.572,8.7,7.7,.241,116.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAL,33,36,30,39,138,98.9,.665,13.6,41.5,.187,139.5,-6.0,-101.0,-257.0,226.0,-108.0,226.0,-102.0,-3.5,-110.0,-195.0,116.5,-110.0,116.5,-110.0,-2.0,178.0,128.0,113.0,128.0,113.0,-148.0,-2.5,-115.0,-185.0,58.0,-115.0,58.0,-105.0,-1.5,-110.0,-135.0,58.0,-115.0,58.0,-105.0,1.5,-135.0,110.0,55.5,103.0,55.5,-123.0,-1.0,-110.0,-135.0,54.5,-105.0,54.5,-115.0


,1,2,3,4,T,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
MIN,23,22,22,24,91,106.7,.398,13.5,29.3,.087,85.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAL,40,27,36,24,127,106.7,.628,14.8,13.9,.156,119.0,-10.5,-103.0,-475.0,229.0,-107.0,229.0,-103.0,-6.0,-105.0,-270.0,116.5,-110.0,116.5,-110.0,-1.0,122.0,100.0,113.0,105.0,113.0,-125.0,-3.5,-110.0,-250.0,58.5,-105.0,58.5,-115.0,-2.0,-115.0,-185.0,57.5,-110.0,57.5,-110.0,-0.5,105.0,-110.0,57.5,105.0,57.5,-125.0,-1.5,-110.0,NaN,55.0,-115.0,55.0,-105.0


,1,2,3,4,T,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
POR,22,36,26,31,115,99.9,.554,8.4,16.7,.141,115.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAL,30,24,31,22,107,99.9,.529,12.9,13.6,.172,107.1,-6.0,-105.0,-215.0,229.0,-108.0,229.0,-102.0,-3.0,-115.0,-175.0,114.0,-110.0,114.0,-110.0,-6.5,115.0,-265.0,118.5,105.0,118.5,-125.0,-2.0,-110.0,-160.0,57.0,-105.0,57.0,-115.0,-1.0,-110.0,-149.0,56.5,100.0,56.5,-120.0,-3.5,100.0,-220.0,58.0,105.0,58.0,-125.0,-1.0,-115.0,NaN,57.5,-115.0,57.5,-105.0


,1,2,3,4,T,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
LAL,35,28,29,29,121,99.4,.646,15.8,17.9,.183,121.7,-7.5,-105.0,-290.0,231.5,-105.0,231.5,-105.0,-4.0,-105.0,-210.0,117.5,-110.0,117.5,-110.0,-1.5,-102.0,-130.0,113.5,-103.0,113.5,-117.0,-2.5,-105.0,-175.0,58.5,-110.0,58.5,-110.0,-1.0,-110.0,-149.0,57.5,-115.0,57.5,-105.0,-1.0,-110.0,-130.0,56.5,-110.0,56.5,-110.0,-1.0,-115.0,NaN,56.0,-110.0,56.0,-110.0
SAS,27,26,29,25,107,99.4,.517,9.3,14.9,.195,107.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


...

,1,2,3,4,T,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
GSW,22,26,33,34,115,100.3,.575,11.9,11.6,.172,114.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LAL,34,30,28,21,113,100.3,.536,14.1,30.8,.289,112.6,-8.5,-108.0,-380.0,227.0,-103.0,227.0,-107.0,-5.0,-115.0,-260.0,116.0,-110.0,116.0,-110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.0,-115.0,-220.0,57.5,-110.0,57.5,-110.0,-2.0,-110.0,-167.0,57.0,-110.0,57.0,-110.0,-2.5,-110.0,NaN,55.5,-110.0,55.5,-110.0,-1.5,-110.0,NaN,54.5,-115.0,54.5,-105.0


,1,2,3,4,T,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
LAL,29,34,23,27,113,98.6,.574,9.6,12.8,.136,114.6,1.5,-103.0,107.0,227.5,-107.0,227.5,-103.0,0.5,-110.0,105.0,117.0,-110.0,117.0,-110.0,6.0,-165.0,150.0,111.5,-110.0,111.5,-110.0,0.5,-115.0,100.0,58.5,-110.0,58.5,-110.0,0.0,-110.0,-111.0,58.0,-110.0,58.0,-110.0,2.0,-130.0,130.0,56.0,105.0,56.0,-125.0,0.0,-110.0,NaN,55.5,-110.0,55.5,-110.0
MIL,33,24,24,25,106,98.6,.557,13.2,13.5,.103,107.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,1,2,3,4,T,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,Spread,Spread Odds,ML,Over,Over Odds,Under,Under Odds,1H Spread,1H Spread Odds,1H ML,1H Over,1H Over Odds,1H Under,1H Under Odds,2H Spread,2H Spread Odds,2H ML,2H Over,2H Over Odds,2H Under,2H Under Odds,1Q Spread,1Q Spread Odds,1Q ML,1Q Over,1Q Over Odds,1Q Under,1Q Under Odds,2Q Spread,2Q Spread Odds,2Q ML,2Q Over,2Q Over Odds,2Q Under,2Q Under Odds,3Q Spread,3Q Spread Odds,3Q ML,3Q Over,3Q Over Odds,3Q Under,3Q Under Odds,4Q Spread,4Q Spread Odds,4Q ML,4Q Over,4Q Over Odds,4Q Under,4Q Under Odds
LAL,29,34,20,18,101,101.4,.481,16.3,12.5,.316,99.6,-10.0,-105.0,-475.0,227.0,-109.0,227.0,-101.0,-6.0,-105.0,-290.0,116.5,-110.0,116.5,-110.0,2.5,-123.0,130.0,110.5,-103.0,110.5,-117.0,-3.0,-115.0,-225.0,57.5,-120.0,57.5,100.0,-2.0,-110.0,-175.0,57.5,-110.0,57.5,-110.0,1.0,-110.0,120.0,55.5,-110.0,55.5,-110.0,-1.5,-110.0,NaN,55.0,-110.0,55.0,-110.0
CHI,17,16,33,24,90,101.4,.440,16.1,21.3,.190,88.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


'LAL covered 10 out of 17 games'

'LAL hit the over on 6 out of 17 games'